In [79]:
import os
from bs4 import BeautifulSoup
import nltk
#nltk.download('punkt')
from nltk.corpus import stopwords
from keybert import KeyBERT
import json

In [80]:
import json
import csv
import os

def get_file_2(path_file):  
  #print(path_file)
  with open(path_file) as f:
      data = json.load(f)
  #print(data.keys())
  dict_text={key: data[key] for key in data.keys() if key in {'title','summary','links'} }
  try:
    dict_text['keyword']=data['keyword_frequency_kebert']
  except:
    print(data.keys())
  return dict_text,data['ground_truth']

def get_class(train_path):
  #print(train_path)
  if os.path.exists(train_path) and os.path.isdir(train_path):
    carpetas = [nombre for nombre in os.listdir(train_path) if os.path.isdir(os.path.join(train_path, nombre))]
    return carpetas
  else:
    print("El directorio especificado no existe o no es un directorio válido.")

def get_archives(train_path):
  if os.path.exists(train_path) and os.path.isdir(train_path):
    carpetas = [os.path.join(train_path,nombre) for nombre in os.listdir(train_path) if os.path.isfile(os.path.join(train_path, nombre))]
    return carpetas
  else:
    print("El directorio especificado no existe o no es un directorio válido.")

def get_all_files(path_data,max_cont=-1):

    #'./data/splits/train_json'
    classes=get_class(path_data)
    archiver_per_clases={}
    prompt_per_clases={}
    cantidad_archivos={}
    promt_all_reson=[]
    archiver_all_reson=[]
    if len(classes)>0:
      for class_name in classes:
        #print(get_archives(os.path.join(path_data,i)))
        archiver_per_clases[class_name]=get_archives(os.path.join(path_data,class_name))[0:max_cont]
        prompt_per_clases[class_name]=[get_file_2(archive) for archive in  archiver_per_clases[class_name]]
        promt_all_reson.extend(prompt_per_clases[class_name])
        archiver_all_reson.extend(archiver_per_clases[class_name])

        #print(len(archiver_per_clases[class_name]),promt_all_reson)
        cantidad_archivos[class_name]=len(archiver_per_clases[class_name])
    else:
       archiver_per_clases['test']=get_archives(os.path.join(path_data,class_name))
       prompt_per_clases['test']=[get_file_2(archive) for archive  in  archiver_per_clases['test']]

    # Clave especial para ir de último
    clave_especial = 'other'
    # Ordenar el diccionario por el valor, manteniendo 'other' al final si existe
    ordenado = sorted(cantidad_archivos.items(), key=lambda item: (item[0] == clave_especial, -item[1]))

    ordenado =[i[0] for i in ordenado]
    return archiver_per_clases,prompt_per_clases,promt_all_reson,archiver_all_reson,ordenado
    

In [85]:
def tranform_text(text):
  return ' '.join([str(text[key]) for key in text.keys() if not key in {'links','keyword'}])

In [86]:
train,promt_train,b_2,d_2,_=get_all_files('../data/splits/train_new_data',-1)
for key in promt_train.keys():
    all_page_texts=[tranform_text(i[0]) for i in promt_train[key]]
    #Se usa keybert para obtener las palabras clave de todos los textos.
    modelo = KeyBERT()
    palabras_clave = modelo.extract_keywords(all_page_texts, stop_words='english')

        
    todas_palabras_clave = [clave for lista_p_claves in palabras_clave for clave in lista_p_claves]

    # Se usa un diccionario para poder calcular la suma del valor de cada palabra clave cuando se encuentra varias veces.
    sums = {}
    
    # Se itera sobre las palabras clave encontradas y se suman sus valores
    for clave in todas_palabras_clave:
        palabra, valor = clave  
        if palabra in sums:
            sums[palabra] += valor
        else:
            sums[palabra] = valor
    
    # Se ordenan las palabras clave encontradas por el resultado de sus sumas de manera descendente
    suma_ordenada = sorted(sums.items(), key=lambda x: x[1], reverse=True)
    
    # Se crea una lista para todas las palabras clave
    palabras_clave_lista = [palabra for palabra, suma in suma_ordenada]
    
    # Ruta de la carpeta de salida para el archivo JSON
    output_folder = f"./key_promt/{key}"
    os.makedirs(output_folder, exist_ok=True)
    
    # Ruta completa al archivo JSON en la carpeta de salida
    output_json_file = os.path.join(output_folder, "keywords.json")
    
    # Se crea un diccionario para almacenar las palabras y el ground_truth
    data_json = {"words": palabras_clave_lista, "ground_truth": key}
    
    # Se guarda el diccionario en un archivo JSON 
    with open(output_json_file, 'w', encoding='utf-8') as json_file:
        json.dump(data_json, json_file, ensure_ascii=False, separators=(',', ':'))
    
    print(f"La lista de palabras clave se ha guardado en '{output_json_file}'")


La lista de palabras clave se ha guardado en './key_promt/staff/keywords.json'
La lista de palabras clave se ha guardado en './key_promt/project/keywords.json'
La lista de palabras clave se ha guardado en './key_promt/other/keywords.json'
La lista de palabras clave se ha guardado en './key_promt/faculty/keywords.json'
La lista de palabras clave se ha guardado en './key_promt/department/keywords.json'
La lista de palabras clave se ha guardado en './key_promt/student/keywords.json'
La lista de palabras clave se ha guardado en './key_promt/course/keywords.json'


In [87]:
#!pip install gensim

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
DEPRECATION: celery 5.0.5 has a non-standard dependency specifier pytz>dev. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [65]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
import gensim
import string

# Descargar stopwords de NLTK
nltk.download('stopwords')
nltk.download('wordnet')

# Tus textos
textos = [
    "Este es el primer texto de ejemplo.",
    "Aquí tienes otro ejemplo de texto.",
    "Este será el tercer texto de muestra.",
    # Agrega más textos según sea necesario
]

# Preprocesamiento
stop = set(stopwords.words('english'))  # Ajusta el idioma de acuerdo a tus textos
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def limpiar_documento(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalizado = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalizado

doc_limpio = [limpiar_documento(doc).split() for doc in textos]

# Crear diccionario y matriz de términos del documento
diccionario = corpora.Dictionary(doc_limpio)
doc_term_matrix = [diccionario.doc2bow(doc) for doc in doc_limpio]

# Crear el modelo LDA
ldamodel = gensim.models.ldamodel.LdaModel(doc_term_matrix, num_topics=3, id2word = diccionario, passes=50)

# Mostrar los tópicos
print(ldamodel.print_topics(num_topics=3, num_words=3))

# Identificar el texto que mejor representa el tópico más común
topicos = ldamodel.get_document_topics(doc_term_matrix)
topicos_principales = [max(tp, key=lambda item: item[1])[0] for tp in topicos]

# Asignar cada texto a su tópico principal
textos_topicos = zip(enumerate(textos), topicos_principales)

# Encontrar el texto más representativo para el tópico más común
#topico_mas_comun = max(set(topicos_principales), key = topicos_principales.count)
#textos_representativos = [texto for texto, topico in textos_topicos if topico == topico_mas_comun ]


umbral_relevancia = 0.5  # Solo considerar textos donde el tópico común represente al menos el 50% 




[(0, '0.130*"ejemplo" + 0.129*"texto" + 0.129*"de"'), (1, '0.127*"el" + 0.127*"este" + 0.127*"texto"'), (2, '0.077*"ejemplo" + 0.077*"texto" + 0.077*"de"')]


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alourido/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/alourido/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
train,promt_train,b_2,d_2,_=get_all_files('../data/splits/train_new_data',-1)

In [4]:
print(_)

['student', 'faculty', 'course', 'project', 'department', 'staff', 'other']


In [5]:
print(train.keys())

dict_keys(['staff', 'project', 'other', 'faculty', 'department', 'student', 'course'])


In [94]:
def get_most_representative_text(promt,num_topics=3,num_words=5):
    # Tus textos
    textos =promt
    
    # Preprocesamiento
    stop = set(stopwords.words('english'))  # Ajusta el idioma de acuerdo a tus textos
    exclude = set(string.punctuation)
    lemma = WordNetLemmatizer()
    
    def limpiar_documento(doc):
        stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        normalizado = " ".join(lemma.lemmatize(word) for word in punc_free.split())
        return normalizado
    
    doc_limpio = [limpiar_documento(doc).split() for doc in textos]
    
    # Crear diccionario y matriz de términos del documento
    diccionario = corpora.Dictionary(doc_limpio)
    doc_term_matrix = [diccionario.doc2bow(doc) for doc in doc_limpio]
    
    # Crear el modelo LDA
    ldamodel = gensim.models.ldamodel.LdaModel(doc_term_matrix, num_topics=3, id2word = diccionario, passes=50)
    
    # Mostrar los tópicos
    print(ldamodel.print_topics(num_topics=num_topics, num_words=num_words))
    
    # Identificar el texto que mejor representa el tópico más común
    topicos = ldamodel.get_document_topics(doc_term_matrix)
    topicos_principales = [max(tp, key=lambda item: item[1])[0] for tp in topicos]
    
    # Asignar cada texto a su tópico principal
    textos_topicos = zip(textos, topicos_principales)
    
    # Encontrar el texto más representativo para el tópico más común
    topico_mas_comun = max(set(topicos_principales), key = topicos_principales.count)
    #textos_representativos = [texto for texto, topico in textos_topicos if topico == topico_mas_comun]
        
    umbral_relevancia = 0.5  # Solo considerar textos donde el tópico común represente al menos el 50% 
    
    # Crear tuplas con el índice, el texto, el tópico principal y la proporción del tópico principal
    textos_topicos_indices = [(indice, texto, topico, prop) for indice, (texto, tp) in enumerate(zip(textos, topicos)) for topico, prop in [max(tp, key=lambda item: item[1])]]
    
    # Ahora el desempaquetado debería funcionar correctamente
    textos_representativos_indices = [(indice,texto,prop) for indice, texto, topico, prop in textos_topicos_indices if topico == topico_mas_comun and prop >= umbral_relevancia]
    
    return sorted(textos_representativos_indices, key=lambda item: item[2], reverse=True)

In [95]:
promt=[tranform_text(i[0]) for i in promt_train['student']]
student_text=get_most_representative_text(promt)
print(student_text,len(student_text))

[(0, '0.023*"computer" + 0.020*"science" + 0.017*"university" + 0.014*"gmt" + 0.012*"page"'), (1, '0.028*"page" + 0.028*"home" + 0.024*"gmt" + 0.023*"1996" + 0.013*"nov"'), (2, '0.030*"page" + 0.025*"gmt" + 0.024*"home" + 0.019*"1996" + 0.014*"server"')]
[(595, " Doug's Page  The title of my M.S. | Web Resources | Vanderbilt | Other Local | Reference Religion | Macintosh | Entertainment | Commercial | Other Computer Science Links AI Links Unified CS Tech Reports Top of page | Top of links Web Resources Navigational Aids Lycos Search Engine Webcrawler Best of the Web Cool Sites The Web as a Teaching Tool Starting Point Yahoo WWW Resources Planet Earth Virtual Library EINet Galaxy City.Net Authoring Tools NCSA Tutorials Web Developer's Virtual Library A Beginner's Guide to HTML Basic Guide for Forms CGI Form Handling in Perl Texture Land Other Web Resoures Netsurfer Digest Top of page | Top of links Vanderbilt Links CS Department Engineering School Vanderbilt University Vanderbilt Univer

In [96]:
#print(train['staff'][student_text[0][0]])
print(train['student'][student_text[0][0]],student_text[0][0])
print(train['student'][student_text[0][0]],student_text[0][0],len(str(promt_train['student'][student_text[0][0]])))
print(train['student'][student_text[1][0]],student_text[1][0],len(str(promt_train['student'][student_text[1][0]])))
print(train['student'][student_text[2][0]],student_text[2][0],len(str(promt_train['student'][student_text[2][0]])))

../data/splits/train_new_data/student/frvyzt.json 595
../data/splits/train_new_data/student/frvyzt.json 595 3628
../data/splits/train_new_data/student/snjbha.json 596 3652
../data/splits/train_new_data/student/awniah.json 476 2156


In [97]:
promt=[tranform_text(i[0]) for i in promt_train['department']]
student_department=get_most_representative_text(promt)
print(student_department,len(student_department))

[(0, '0.042*"science" + 0.036*"computer" + 0.023*"university" + 0.023*"department" + 0.015*"page"'), (1, '0.040*"department" + 0.034*"science" + 0.030*"computer" + 0.021*"university" + 0.016*"1996"'), (2, '0.041*"science" + 0.041*"computer" + 0.027*"department" + 0.016*"page" + 0.016*"c"')]
[(126, "Department of Computer Science Home Page Date: Tue, 26 Nov 1996 22:57:02 GMT Server: NCSA/1.5.1 Last-modified: Wed, 20 Nov 1996 20:41:27 GMT Content-type: text/html Content-length: 3833 Department of Computer Science Home Page Rochester Institute of Technology Department of Computer Science 102 Lomb Memorial Drive Rochester, NY 14623-5608 Email: csdept@cs.rit.edu Phone:  +1.716.475.2995 Fax:  +1.716.475.7100 The following information is available about the Department: Computer Science Programs of Study Computer Science Faculty Faculty Research Interests Student Services Support Our Anonymous FTP Site Frequently Asked Questions Lab Reference and Information RIT Co-op Office and Other Job Serv

In [98]:
#print(train['staff'][student_text[0][0]])
print(train['department'][student_department[0][0]],student_department[0][0])
print(train['department'][student_department[0][0]],student_department[0][0],len(str(promt_train['department'][student_department[0][0]])))
print(train['department'][student_department[1][0]],student_department[1][0],len(str(promt_train['department'][student_department[1][0]])))
print(train['department'][student_department[2][0]],student_department[2][0],len(str(promt_train['department'][student_department[2][0]])))

../data/splits/train_new_data/department/tgnfag.json 126
../data/splits/train_new_data/department/tgnfag.json 126 1989
../data/splits/train_new_data/department/oqeazcb.json 117 2019
../data/splits/train_new_data/department/zujfjnjr.json 6 1981


In [99]:
promt=[tranform_text(i[0]) for i in promt_train['faculty']]
student_faculty=get_most_representative_text(promt)
print(student_faculty,len(student_faculty))

[(0, '0.027*"gmt" + 0.021*"1996" + 0.020*"computer" + 0.019*"science" + 0.015*"server"'), (1, '0.012*"computer" + 0.008*"software" + 0.008*"system" + 0.008*"university" + 0.007*"page"'), (2, '0.016*"system" + 0.012*"computer" + 0.010*"science" + 0.010*"research" + 0.006*"page"')]
[(602, "Harriet Fell's Home Page MIME-Version: 1.0 Server: CERN/3.0 Date: Thursday, 21-Nov-96 19:21:55 GMT Content-Type: text/html Content-Length: 5160 Last-Modified: Friday, 01-Nov-96 15:59:25 GMT Harriet Fell's Home Page Harriet Fell's Home Page [ SIGCAPH - SIGCSE | Research | Courses | Family | Miscellaneous ] Click here for the home pages for Northeastern University and the College of Computer Science . ACM SIGCAPH and SIGSCE Information ACM SIGCAPH - Association for Computing Machinery Special Interest Group in Computers and the Physically Handicapped ASCII Versions of ASSETS '96 Papers ACM SIGCSE/SIGCUE Conference on Integrating Technology into Computer Science Education Barcelona, Spain, June 2 - 6, 199

In [100]:
#print(train['staff'][student_text[0][0]])
print(train['faculty'][student_faculty[0][0]],student_faculty[0][0])
print(train['faculty'][student_faculty[0][0]],student_faculty[0][0],len(str(promt_train['faculty'][student_faculty[0][0]])))
print(train['faculty'][student_faculty[1][0]],student_faculty[1][0],len(str(promt_train['faculty'][student_faculty[1][0]])))
print(train['faculty'][student_faculty[2][0]],student_faculty[2][0],len(str(promt_train['faculty'][student_faculty[2][0]])))

../data/splits/train_new_data/faculty/anbzhw.json 602
../data/splits/train_new_data/faculty/anbzhw.json 602 2178
../data/splits/train_new_data/faculty/epsinrn.json 479 1756
../data/splits/train_new_data/faculty/wnjlqjq.json 110 1534


In [101]:
promt=[tranform_text(i[0]) for i in promt_train['course']]
student_course=get_most_representative_text(promt)
print(student_course,len(student_course))

[(0, '0.024*"gmt" + 0.024*"1996" + 0.019*"computer" + 0.015*"course" + 0.014*"server"'), (1, '0.016*"page" + 0.016*"c" + 0.015*"1996" + 0.013*"assignment" + 0.011*"homework"'), (2, '0.010*"c" + 0.007*"language" + 0.007*"programming" + 0.006*"course" + 0.006*"software"')]
[(444, 'CS132 Using Computers - Lectures 3 & 4 Date: Mon, 11 Nov 1996 19:03:38 GMT Server: NCSA/1.5 Content-type: text/html Last-modified: Wed, 23 Oct 1996 03:16:49 GMT Content-length: 5893 CS132 Using Computers - Lectures 3 & 4 CS132 - Using Computers Instructor Info: Instructor: Sally Peterson Office: 5381 Computer Science Phone: 263-7763 E-Mail: sally@cs.wisc.edu or slpeters@facstaff.wisc.edu Office Hours: Tuesday 10:45-11:45, Thursday 11:00-12:00 or by appointment Vital Class Info: Time: TR 8:00 to 9:15 (Lecture 3) TR 9:30 to 10:45 (Lecture 4) Place: All lectures held in 1800 Engineering Hall Lecture Text: Information Technology and Society by Laudon, Traver & Laudon Lab Text: Point, Click & Drag: Using the Macinto

In [102]:
#print(train['staff'][student_text[0][0]])
print(train['course'][student_course[0][0]],student_course[0][0],len(str(promt_train['course'][student_course[0][0]])))
print(train['course'][student_course[1][0]],student_course[1][0],len(str(promt_train['course'][student_course[1][0]])))
print(train['course'][student_course[2][0]],student_course[2][0],len(str(promt_train['course'][student_course[2][0]])))
print(train['course'][student_course[3][0]],student_course[3][0],len(str(promt_train['course'][student_course[3][0]])))

../data/splits/train_new_data/course/iaxei.json 444 2086
../data/splits/train_new_data/course/bvfjoth.json 517 2086
../data/splits/train_new_data/course/vyvfn.json 682 2086
../data/splits/train_new_data/course/lvmecx.json 584 1383


In [103]:
len(str(promt_train['course'][student_course[0][0]]))

2086

In [104]:
promt=[tranform_text(i[0]) for i in promt_train['project']]

student_project=get_most_representative_text(promt)
print(student_project,len(student_project))

[(0, '0.015*"gmt" + 0.014*"research" + 0.014*"computer" + 0.013*"group" + 0.012*"1996"'), (1, '0.018*"system" + 0.016*"gmt" + 0.013*"research" + 0.013*"laboratory" + 0.012*"1996"'), (2, '0.015*"gmt" + 0.015*"project" + 0.009*"group" + 0.009*"1996" + 0.009*"server"')]
[(131, 'TAMU Robotics Group home page. Date: Tue, 26 Nov 1996 00:46:47 GMT Server: Apache/1.1.1 Content-type: text/html Content-length: 5866 Last-modified: Fri, 08 Mar 1996 15:28:53 GMT TAMU Robotics Group home page. Further Information: Research projects Publications Personnel Universities Space Automation/Robotics Consortium (USARC) Center for Fuzzy Logic, Robotics, and Intelligent Systems Links to other robotics stuff: Robotics Internet Resources Page Links to companies, government labs Collision detection software Randy Wilson at Sandia comp.robotics Frequently Asked Questions List IEEE Robotics and Automation Society Robotics, AI, and Control Conference List Stanford Dexterous Manipulation Lab NASA JSC Automation and 

In [105]:
#print(train['staff'][student_text[0][0]])
print(train['project'][student_project[0][0]],student_project[0][0])

../data/splits/train_new_data/project/wlcas.json 131


In [106]:
promt=[tranform_text(i[0]) for i in promt_train['staff']]

student_staff=get_most_representative_text(promt)
print(student_staff,len(student_staff))

[(0, '0.019*"gmt" + 0.018*"computer" + 0.016*"1996" + 0.016*"science" + 0.012*"research"'), (1, '0.023*"home" + 0.023*"page" + 0.022*"gmt" + 0.017*"1996" + 0.014*"server"'), (2, '0.013*"page" + 0.008*"web" + 0.007*"system" + 0.006*"database" + 0.006*"science"')]
[(61, 'Home Page for Thomas Wallis Date: Mon, 25 Nov 1996 23:29:38 GMT Server: Apache/1.0.3 Content-type: text/html Content-length: 1703 Last-modified: Tue, 12 Nov 1996 16:44:59 GMT Home Page for Thomas Wallis Home Page for Thomas Wallis Thomas Wallis System Manager Computer Science Department Wichita State University Wichita , Kansas 67260-0083 Schedule Office Hours: M-F 8-5 Office: 213 SB E-Mail Address: wallis@cs.twsu.edu Phone: 316-978-3921 CS Department UNIX System Rules Handin Program Description for Graders and Students (BETA) *UPDATED* New Feature: Ask the System Manager Back Email Tom Wallis Here The information accessible from this server is provided in good faith, but neither the Wichita State University, nor the Com

In [107]:
#print(train['staff'][student_text[0][0]])
print(train['staff'][student_staff[0][0]],student_staff[0][0],len(str(promt_train['staff'][student_staff[0][0]])))
print(train['staff'][student_staff[1][0]],student_staff[1][0],len(str(promt_train['staff'][student_staff[1][0]])))
print(train['staff'][student_staff[2][0]],student_staff[2][0],len(str(promt_train['staff'][student_staff[2][0]])))
print(train['staff'][student_staff[3][0]],student_staff[3][0],len(str(promt_train['staff'][student_staff[3][0]])))

../data/splits/train_new_data/staff/xdciwe.json 61 1109
../data/splits/train_new_data/staff/ullwnir.json 76 1093
../data/splits/train_new_data/staff/torsffaj.json 64 1118
../data/splits/train_new_data/staff/dhrhl.json 103 833


In [108]:
promt=[tranform_text(i[0]) for i in promt_train['other']]
student_other=get_most_representative_text(promt)
print(student_other,len(student_other))


[(0, '0.013*"slide" + 0.005*"university" + 0.005*"authannot" + 0.005*"program" + 0.003*"david"'), (1, '0.093*"000" + 0.043*"001" + 0.025*"1" + 0.013*"002" + 0.013*"13"'), (2, '0.020*"gmt" + 0.013*"1996" + 0.013*"server" + 0.012*"date" + 0.011*"contenttype"')]
[(468, 'EOSDIS Bookmarks of vbb MIME-Version: 1.0 Server: CERN/3.0 Date: Tuesday, 07-Jan-97 14:44:02 GMT Content-Type: text/html Content-Length: 34339 Last-Modified: Saturday, 21-Sep-96 03:55:56 GMT EOSDIS Bookmarks of vbb <!-- Compiled by V. B. Balayoghan (vbb@cs.utexas.edu)> <!--BODY BACKGROUND = "images/dg64-0105.gif" TEXT = "#00ff00" LINK = "#ffff22" VLINK = "#66bb22" ALINK = "#ff0000"> A Catalog of EOSDIS Related Information Table of Contents EOSDIS Sites EOSDIS Documents People and Contacts Geographic Data and Metadata Standards Other Data and Protocol Standards Databases and Multiresolution Miscellany <!--IMG SRC="images/pat_crossed2.gif" ALT="-----"> <!--IMG SRC="/images/stanford/line.dash.gif" ALT="-----"> EOSDIS Sites NA

In [109]:
print(train['other'][student_other[0][0]],student_other[0][0],len(str(promt_train['other'][student_other[0][0]])))
print(train['other'][student_other[1][0]],student_other[1][0],len(str(promt_train['other'][student_other[1][0]])))
print(train['other'][student_other[2][0]],student_other[2][0],len(str(promt_train['other'][student_other[2][0]])))
print(train['other'][student_other[3][0]],student_other[3][0],len(str(promt_train['other'][student_other[3][0]])))
print(train['other'][student_other[4][0]],student_other[4][0],len(str(promt_train['other'][student_other[4][0]])))
print(train['other'][student_other[5][0]],student_other[5][0],len(str(promt_train['other'][student_other[5][0]])))
print(train['other'][student_other[6][0]],student_other[6][0],len(str(promt_train['other'][student_other[6][0]])))

../data/splits/train_new_data/other/mqwloql.json 468 7469
../data/splits/train_new_data/other/cnkdwoo.json 1006 5156
../data/splits/train_new_data/other/jwdptqg.json 1459 4587
../data/splits/train_new_data/other/ghczpf.json 2469 4037
../data/splits/train_new_data/other/efvwrhbq.json 2441 4391
../data/splits/train_new_data/other/fnmidivq.json 1681 3330
../data/splits/train_new_data/other/iprwhcrm.json 312 3729


In [110]:

for key in promt_train.keys():
    promt=[tranform_text(i[0]) for i in promt_train[key]]
    student_text=get_most_representative_text(promt,20,20)
    for i in range(0,8):
        print(train[key][student_text[i][0]],' ind: ',student_text[i][0],
              ' val: ',student_text[i][2], ' tamno: ',len(str(promt_train[key][student_text[i][0]])))


[(0, '0.023*"page" + 0.021*"gmt" + 0.019*"home" + 0.017*"1996" + 0.012*"research" + 0.012*"server" + 0.012*"computer" + 0.011*"science" + 0.011*"university" + 0.011*"date" + 0.011*"texthtml" + 0.011*"contenttype" + 0.010*"nov" + 0.010*"lastmodified" + 0.010*"contentlength" + 0.009*"tue" + 0.009*"department" + 0.007*"26" + 0.004*"system" + 0.004*"rice"'), (1, '0.021*"gmt" + 0.018*"computer" + 0.015*"1996" + 0.015*"science" + 0.012*"page" + 0.012*"server" + 0.011*"home" + 0.011*"date" + 0.011*"texthtml" + 0.011*"contenttype" + 0.010*"lastmodified" + 0.010*"university" + 0.010*"contentlength" + 0.010*"system" + 0.009*"department" + 0.009*"nov" + 0.007*"research" + 0.007*"tue" + 0.005*"new" + 0.005*"mon"'), (2, '0.007*"page" + 0.007*"computer" + 0.006*"m" + 0.006*"science" + 0.005*"web" + 0.005*"department" + 0.005*"java" + 0.005*"university" + 0.004*"home" + 0.004*"database" + 0.004*"site" + 0.004*"paradise" + 0.004*"1996" + 0.003*"office" + 0.003*"computing" + 0.003*"system" + 0.003*"im"

In [111]:

for key in promt_train.keys():
    promt=[tranform_text(i[0]) for i in promt_train[key]]
    student_text=get_most_representative_text(promt,10,20)
    for i in range(0,8):
        print(train[key][student_text[i][0]],' ind: ',student_text[i][0],
              ' val: ',student_text[i][2], ' tamno: ',len(str(promt_train[key][student_text[i][0]])))

[(0, '0.010*"research" + 0.005*"c" + 0.005*"science" + 0.005*"b" + 0.004*"system" + 0.004*"page" + 0.004*"web" + 0.004*"r" + 0.004*"york" + 0.004*"computer" + 0.004*"new" + 0.003*"group" + 0.003*"processing" + 0.003*"danilo" + 0.003*"john" + 0.003*"program" + 0.003*"also" + 0.003*"computing" + 0.003*"associate" + 0.003*"student"'), (1, '0.026*"gmt" + 0.024*"1996" + 0.018*"page" + 0.015*"nov" + 0.015*"server" + 0.014*"computer" + 0.013*"date" + 0.013*"contenttype" + 0.013*"texthtml" + 0.013*"science" + 0.013*"contentlength" + 0.013*"lastmodified" + 0.012*"tue" + 0.012*"university" + 0.012*"home" + 0.012*"department" + 0.011*"research" + 0.009*"26" + 0.006*"apache111" + 0.005*"office"'), (2, '0.020*"home" + 0.020*"page" + 0.018*"gmt" + 0.016*"computer" + 0.014*"science" + 0.010*"university" + 0.010*"server" + 0.010*"system" + 0.010*"1996" + 0.009*"date" + 0.009*"contenttype" + 0.009*"texthtml" + 0.009*"lastmodified" + 0.008*"contentlength" + 0.008*"department" + 0.006*"new" + 0.006*"nov"

In [ ]:
import gensim
from gensim import corpora
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

# Preprocesamiento y funciones auxiliares

def limpiar_documento(doc):
    stop = set(stopwords.words('english'))  # Ajusta el idioma de acuerdo a tus textos
    exclude = set(string.punctuation)
    lemma = WordNetLemmatizer()
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalizado = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalizado

def get_most_representative_text(texts, num_topics=3, num_words=5):
    doc_limpio = [limpiar_documento(doc).split() for doc in texts]
    diccionario = corpora.Dictionary(doc_limpio)
    doc_term_matrix = [diccionario.doc2bow(doc) for doc in doc_limpio]
    ldamodel = gensim.models.ldamodel.LdaModel(doc_term_matrix, num_topics=num_topics, id2word = diccionario, passes=50)
    print(ldamodel.print_topics(num_topics=num_topics, num_words=num_words))
    topicos = ldamodel.get_document_topics(doc_term_matrix)
    topicos_principales = [max(tp, key=lambda item: item[1])[0] for tp in topicos]
    topico_mas_comun = max(set(topicos_principales), key = topicos_principales.count)
    umbral_relevancia = 0.5  
    textos_topicos_indices = [(indice, texto, topico, prop) for indice, (texto, tp) in enumerate(zip(texts, topicos)) for topico, prop in [max(tp, key=lambda item: item[1])]]
    textos_representativos_indices = [(indice, texto, prop) for indice, texto, topico, prop in textos_topicos_indices if topico == topico_mas_comun and prop >= umbral_relevancia]
    return sorted(textos_representativos_indices, key=lambda item: item[2], reverse=True)

def calculate_distance(textos_representativos, textos_clases):
    vectorizador = TfidfVectorizer()
    textos_todos = [texto for clase in textos_clases for texto in clase]
    vectorizador.fit(textos_todos)
    resultados=[]
    distancias_por_clase = {}
    for indice,texto_representativo, prop in textos_representativos:
        texto_representativo_vectorizado = vectorizador.transform([texto_representativo]).toarray()
        
        distancias_por_texto = {clase: [] for clase in range(len(textos_clases))}
        for clase, textos in enumerate(textos_clases):
            for texto in textos:
                texto_vectorizado = vectorizador.transform([texto]).toarray()
                similitud = cosine_similarity(texto_representativo_vectorizado, texto_vectorizado)[0][0]
                distancias_por_texto[clase].append(1 - similitud)
        
        distancias_promedio = {clase: np.mean(distancias) for clase, distancias in distancias_por_texto.items()}
        #distancias_por_clase[indice] = distancias_promedio
        
        resultados.append((indice, texto_representativo, prop, distancias_promedio))

    return resultados
# Ejemplo de uso
for key in promt_train.keys():
    mis_textos = [tranform_text(i[0]) for i in promt_train[key]]
    mis_textos_clasificados = [[tranform_text(i[0]) for i in promt_train[not_key]] for not_key in promt_train.keys() if not_key != key]

    textos_representativos = get_most_representative_text(mis_textos)
    distancias = calculate_distance(textos_representativos, mis_textos_clasificados)
    
    # Encontrar el texto más distante
    clase_mas_distante = max(distancias, key=distancias.get)
    # Para ver los resultados
    for index_orig, texto, prop, distancias in resultados_distancias[:10]:
        print(f"Texto Representativo {index_orig} (Proporción: {prop}):")
        print(train[key][index_orig],len(str(promt_train[key][index_orig])))
        for clase, distancia in distancias.items():
            print(f"  Distancia a Clase {clase}: {distancia}")
        


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alourido/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/alourido/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


[(0, '0.012*"gmt" + 0.011*"1996" + 0.009*"research" + 0.009*"university" + 0.009*"computer"'), (1, '0.021*"gmt" + 0.018*"1996" + 0.015*"page" + 0.015*"server" + 0.014*"computer"'), (2, '0.019*"page" + 0.015*"home" + 0.014*"computer" + 0.011*"science" + 0.010*"gmt"')]
